In [1]:
import pandas as pd
from prometheus_pandas import query

In [2]:
df = pd.read_csv('Data/Simple_times.csv')
df

,start,finish,message,levelname,delta,source,target,type
0,2023-02-03 23:16:11.226,2023-02-03 23:16:13.369,0 PushEvent created,INFO,2.143,u: fortiz,r: fortiz/sat,PushEvent
1,2023-02-03 23:16:13.369,2023-02-03 23:16:14.559,1 PushEvent created,INFO,1.190,u: CarlosAlbertoFigueiredo,r: CarlosAlbertoFigueiredo/ServidorGit,PushEvent
2,2023-02-03 23:16:14.559,2023-02-03 23:16:15.414,2 PushEvent created,INFO,0.855,u: BernardTatin,r: BernardTatin/ideal-cobol,PushEvent
3,2023-02-03 23:16:15.414,2023-02-03 23:16:16.694,3 PushEvent created,INFO,1.280,u: juliozohar,r: juliozohar/Aleph-Mainframe,PushEvent
4,2023-02-03 23:16:16.694,2023-02-03 23:16:17.563,4 PushEvent created,INFO,0.869,u: dbarelop,r: dbarelop/SLEG-1516,PushEvent
...,...,...,...,...,...,...,...,...
533538,2023-02-05 01:02:10.497,2023-02-05 01:02:10.688,533538 FollowEvent created,INFO,0.191,u: Liampronan,u: lzyc,FollowEvent
533539,2023-02-05 01:02:10.688,2023-02-05 01:02:10.911,533539 FollowEvent created,INFO,0.223,u: das-vinculum,u: Object-Bootcamp,FollowEvent
533540,2023-02-05 01:02:10.911,2023-02-05 01:02:11.079,533540 FollowEvent created,INFO,0.168,u: anthonyheckmann,u: rk13,FollowEvent
533541,2023-02-05 01:02:11.079,2023-02-05 01:02:11.246,533541 FollowEvent created,INFO,0.167,u: artur-augustyniak,u: danielrohers,FollowEvent


In [3]:
p = query.Prometheus('http://localhost:9090')

In [4]:
def select_query(p, metric, init, endt, step):
    
    it = pd.Timestamp(init).strftime('%Y-%m-%dT%H:%M:%SZ')
    et = pd.Timestamp(endt).strftime('%Y-%m-%dT%H:%M:%SZ')
    
    if metric == 'cpu':
        return p.query_range('max(max_over_time(gitlab_sli:gitlab_service_saturation:ratio{component="cpu"}[1m]))',
                  it, 
                  et, 
                  step).mean().iloc[0]
    if metric == 'latency':
        return p.query_range('sum by (instance) (rate(nginx_vts_upstream_request_seconds_total{instance="localhost:8060"}[1m])) \
                  /sum by (instance) (rate(nginx_vts_upstream_requests_total{instance="localhost:8060"}[1m]))',
                  it,
                  et, 
                  step).mean().iloc[0]
        
    if metric == 'memory':
        return p.query_range('clamp_min(clamp_max(1 -((node_memory_MemFree_bytes{instance="localhost:9100"} \
                  + node_memory_Buffers_bytes{instance="localhost:9100"} \
                  + node_memory_Cached_bytes{instance="localhost:9100"}))\
                  /node_memory_MemTotal_bytes{instance="localhost:9100"},1),0)',
                  it,
                  et, 
                  step).mean().iloc[0]
    
    if metric == 'rps':
        return p.query_range('max(avg_over_time(gitlab_sli:code_method_route:workhorse_http_request_count:rate1m[1m]))',
                             it, 
                             et, 
                             step).mean().iloc[0]
    
    

In [5]:
#df = df[df['delta'] > 0]
#df

In [6]:
df.loc[:,'cpu'] = df.apply(lambda x: select_query(p,'cpu', x['start'], x['finish'], x['delta']), axis=1)
df.loc[:,'mem'] = df.apply(lambda x: select_query(p,'memory', x['start'], x['finish'], x['delta']), axis=1)
df.loc[:,'lat'] = df.apply(lambda x: select_query(p,'latency', x['start'], x['finish'], x['delta']), axis=1)
df

,start,finish,message,levelname,delta,source,target,type,cpu,mem,lat
0,2023-02-03 23:16:11.226,2023-02-03 23:16:13.369,0 PushEvent created,INFO,2.143,u: fortiz,r: fortiz/sat,PushEvent,0.048389,0.657806,0.156000
1,2023-02-03 23:16:13.369,2023-02-03 23:16:14.559,1 PushEvent created,INFO,1.190,u: CarlosAlbertoFigueiredo,r: CarlosAlbertoFigueiredo/ServidorGit,PushEvent,0.048389,0.657806,0.156000
2,2023-02-03 23:16:14.559,2023-02-03 23:16:15.414,2 PushEvent created,INFO,0.855,u: BernardTatin,r: BernardTatin/ideal-cobol,PushEvent,0.048389,0.657806,0.156000
3,2023-02-03 23:16:15.414,2023-02-03 23:16:16.694,3 PushEvent created,INFO,1.280,u: juliozohar,r: juliozohar/Aleph-Mainframe,PushEvent,0.048389,0.657806,0.156000
4,2023-02-03 23:16:16.694,2023-02-03 23:16:17.563,4 PushEvent created,INFO,0.869,u: dbarelop,r: dbarelop/SLEG-1516,PushEvent,0.048389,0.672314,0.156000
...,...,...,...,...,...,...,...,...,...,...,...
533538,2023-02-05 01:02:10.497,2023-02-05 01:02:10.688,533538 FollowEvent created,INFO,0.191,u: Liampronan,u: lzyc,FollowEvent,0.517100,0.775360,0.032585
533539,2023-02-05 01:02:10.688,2023-02-05 01:02:10.911,533539 FollowEvent created,INFO,0.223,u: das-vinculum,u: Object-Bootcamp,FollowEvent,0.517100,0.775360,0.032585
533540,2023-02-05 01:02:10.911,2023-02-05 01:02:11.079,533540 FollowEvent created,INFO,0.168,u: anthonyheckmann,u: rk13,FollowEvent,0.517100,0.775360,0.032585
533541,2023-02-05 01:02:11.079,2023-02-05 01:02:11.246,533541 FollowEvent created,INFO,0.167,u: artur-augustyniak,u: danielrohers,FollowEvent,0.517100,0.775360,0.032585


In [7]:
c = df[['source', 'cpu']].groupby('source').mean()

In [8]:
m = df[['source', 'mem']].groupby('source').mean()

In [9]:
l = df[['source', 'lat']].groupby('source').mean()

In [10]:
ds = c.join(m).join(l)
ds = ds.reset_index()
ds

,source,cpu,mem,lat
0,u: 0532,0.427304,0.765748,0.028698
1,u: 0x00,0.425655,0.764083,0.028549
2,u: 12-tp9d,0.426197,0.766558,0.028755
3,u: 234x,0.428646,0.765199,0.028839
4,u: 4148,0.426701,0.764392,0.028709
...,...,...,...,...
1518,u: zenware,0.427865,0.764472,0.028667
1519,u: zgrep,0.537033,0.733931,0.042178
1520,u: zn,0.425222,0.764024,0.028609
1521,u: zsim0n,0.424635,0.763805,0.028591


In [11]:
ds.to_csv('Data/avg_resource_utilisation_simple.csv', index=False)